In [ ]:
# install pyscicat
!pip install 'pyscicat>=0.4.4'

In [ ]:
# create scicat client

from pyscicat.client import ScicatClient
import os

base_url = os.environ.get('BE_BASE_URL')
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

client = ScicatClient(base_url=base_url, username=username, password=password)

In [ ]:
# create (raw) dataset

from pyscicat.model import Ownable, RawDataset
from datetime import datetime

ownable = Ownable(
    ownerGroup="example_research_group", # only obligatory element of this model
    accessGroups=['group1', 'group2'],
    instrumentGroup='instrument1'
)

dataset = RawDataset(
    contactEmail="example@email.net", # must be valid email in format
    creationTime=datetime(year=2024, month=4, day=12, hour=12, minute=35, second=30).isoformat(),
    datasetName="example_dataset",
    description="You can insert a lengthy description here",
    instrumentId="example.instrument.id",
    isPublished=False,
    keywords=["important", "biology", "example"],
    license="Public Domain",
    numberOfFiles = 6,
    orcidOfOwner="0000-0001-5109-3700",
    owner="Emmett Exemplum, PhD",
    ownerEmail="example.exemplum@some_uni.ch",
    size=589824, # in bytes!
    sourceFolder="/datasets/example_dataset", # this will have to reflect the retrieval location for the archival system 
    #sourceFolderHost="earth.net", # same as above but the network host part (instead of filesystem)
    validationStatus="valid",
    version="4.0.0", # optional
    scientificMetadata={}, # optional
    principalInvestigator="Mr. Irvine Investigator",
    creationLocation="University of Example, Exemplia",
    #dataFormat="someformat", # optional
    sampleId="example.sample.id",
    **ownable.model_dump()
)

# attempt to create dataset, *CAN* throw ScicatCommError
dataset_id = client.datasets_create(dataset)

In [ ]:
# Add datablock to dataset

from pyscicat.model import DataFile, CreateDatasetOrigDatablockDto
from datetime import datetime

# disable info level logging
import logging
logging.getLogger().setLevel(logging.FATAL)

files = [
    "file1.txt",
    "someimage.raw",
    "thumbnail/someimage.jpg",
    "thumbnail/FoilHoles.jpg",
    "transformations.json",
    "FoilHoles.raw"
]
file_sizes = [
    100,
    204800,
    20480,
    20480,
    1000,
    342964
]
file_times = [
    datetime(year=2024, month=4, day=12, hour=12, minute=35, second=30),
    datetime(year=2024, month=2, day=4, hour=5, minute=56, second=39),
    datetime(year=2024, month=3, day=30, hour=19, minute=4, second=53),
    datetime(year=2024, month=4, day=2, hour=16, minute=25, second=37),
    datetime(year=2024, month=4, day=12, hour=8, minute=13, second=44),
    datetime(year=2024, month=2, day=4, hour=4, minute=24, second=17)
]
dataFileList = [
    DataFile(path=p, size=s, time=t.isoformat()) for (p, s, t) in zip(files, 
                                                                      file_sizes, 
                                                                      file_times)
]

data_block = CreateDatasetOrigDatablockDto(
    size=576, version=1, dataFileList=dataFileList
)

client.datasets_origdatablock_create(dataset_id, data_block);

In [ ]:
# Upload attachment

from pyscicat.client import encode_thumbnail
from pyscicat.model import Attachment

attachment = Attachment(
    datasetId=dataset_id,
    thumbnail=encode_thumbnail("example_files/thumbnail.png", "png"),
    caption="Example thumbnail image as attachment",
    **ownable.model_dump()
)

client.datasets_attachment_create(attachment);